In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd
import torch
from datasets import load_dataset

In [2]:
!pip install -U datasets
!pip install -U transformers

In [3]:
# Carrega a base SST-2 (parte do benchmark GLUE)
dataset = load_dataset("glue", "sst2")

# Visualiza amostras
print(dataset['train'][0])
print(f"Número de frases de treino: {len(dataset['train'])}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}
Número de frases de treino: 67349


In [4]:
print(dataset)
print("Valores únicos nos rótulos de treino:", set(dataset['train']['label']))
print("Valores únicos nos rótulos de validação:", set(dataset['validation']['label']))
print("Valores únicos nos rótulos de teste:", set(dataset['test']['label']))

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
Valores únicos nos rótulos de treino: {0, 1}
Valores únicos nos rótulos de validação: {0, 1}
Valores únicos nos rótulos de teste: {-1}


In [5]:
train_texts = dataset['train']['sentence']
train_labels = dataset['train']['label']

val_texts = dataset['validation']['sentence']
val_labels = dataset['validation']['label']

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import os

os.makedirs("./logs2", exist_ok=True)
os.makedirs("./meu_modelo", exist_ok=True)

In [10]:
from transformers import Trainer
import torch

class CustomTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Aplicar os pesos das classes
        if self.class_weights is not None:
            # Mover os pesos para o mesmo device do modelo
            class_weights = torch.tensor(self.class_weights).to(logits.device)
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        else:
            loss_fct = torch.nn.CrossEntropyLoss()

        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # pega a classe com maior probabilidade
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [12]:
# Calcular os pesos das classes
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Assumindo que train_labels são seus rótulos de treinamento
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
).astype(np.float32)

# Configurar os argumentos de treinamento normalmente
training_args = TrainingArguments(
    output_dir="./meu_modelo",
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=600,
    learning_rate=1e-5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# Usar o CustomTrainer com os pesos das classes
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    class_weights=class_weights  # Passar os pesos aqui
)

# Iniciar o treinamento
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.225300,0.264883,0.903670,0.910959,0.898649,0.904762
2,0.203700,0.310841,0.905963,0.895197,0.923423,0.909091


TrainOutput(global_step=4210, training_loss=0.22066499978799434, metrics={'train_runtime': 803.3483, 'train_samples_per_second': 167.671, 'train_steps_per_second': 5.241, 'total_flos': 2300086292646384.0, 'train_loss': 0.22066499978799434, 'epoch': 2.0})

In [13]:
trainer.evaluate()


{'eval_loss': 0.31084147095680237,
 'eval_accuracy': 0.9059633027522935,
 'eval_precision': 0.8951965065502183,
 'eval_recall': 0.9234234234234234,
 'eval_f1': 0.9090909090909091,
 'eval_runtime': 1.4637,
 'eval_samples_per_second': 595.739,
 'eval_steps_per_second': 19.129,
 'epoch': 2.0}

In [14]:
model.save_pretrained("./meu_modelo_distilbertBaseSST-2")
tokenizer.save_pretrained("./meu_modelo_distilbertBaseSST-2")

('./meu_modelo_distilbertBaseSST-2/tokenizer_config.json',
 './meu_modelo_distilbertBaseSST-2/special_tokens_map.json',
 './meu_modelo_distilbertBaseSST-2/vocab.txt',
 './meu_modelo_distilbertBaseSST-2/added_tokens.json',
 './meu_modelo_distilbertBaseSST-2/tokenizer.json')